In [1]:
import glob
import os.path as osp
import uproot
import awkward as ak
import torch
import numpy as np
import random
import tqdm
from torch_geometric.data import Data, Dataset

import numpy as np
import subprocess
import tqdm
from tqdm import tqdm
import pandas as pd

import os
import os.path as osp

import glob

import h5py
import uproot

import torch
from torch import nn
from torch_geometric.data import Data
from torch_geometric.data import Dataset
from torch_geometric.data import DataLoader

import awkward as ak
import random
from torch_geometric.nn import knn_graph
import torch.nn.functional as F

In [2]:
import numpy as np
import subprocess
import tqdm
from tqdm import tqdm
import pandas as pd

import os
import os.path as osp

import glob

import h5py
import uproot

import torch
from torch import nn


from torch_geometric.data import Data
from torch_geometric.data import Dataset
from torch_geometric.data import DataLoader

import awkward as ak
import random

#singularity shell --bind /afs/cern.ch/user/p/pkakhand/public/CL/  /afs/cern.ch/user/p/pkakhand/geometricdl.sif

#singularity shell --bind /eos/project/c/contrast/public/solar/  /afs/cern.ch/user/p/pkakhand/geometricdl.sif
#source /cvmfs/sft.cern.ch/lcg/views/LCG_103cuda/x86_64-centos9-gcc11-opt/setup.sh

def find_highest_branch(path, base_name):
    with uproot.open(path) as f:
        # Find keys that exactly match the base_name (not containing other variations)
        branches = [k for k in f.keys() if k.startswith(base_name + ';')]
        
        # Sort and select the highest-numbered branch
        sorted_branches = sorted(branches, key=lambda x: int(x.split(';')[-1]))
        return sorted_branches[-1] if sorted_branches else None

def remove_duplicates(A,B):    
    all_masks = []
    for event_idx, event in enumerate(A):
        flat_A = np.array(ak.flatten(A[event_idx]))
        flat_B = np.array(ak.flatten(B[event_idx]))
        
        # Initialize a mask to keep track of which values to keep
        mask = np.zeros_like(flat_A, dtype=bool)

        # Iterate over the unique elements in A
        for elem in np.unique(flat_A):
            # Get the indices where the element occurs in A
            indices = np.where(flat_A == elem)[0]

            # If there's more than one occurrence, keep the one with the max B value
            if len(indices) > 1:
                max_index = indices[np.argmax(flat_B[indices])]
                mask[max_index] = True
            else:
                # If there's only one occurrence, keep it
                mask[indices[0]] = True

        unflattened_mask = ak.unflatten(mask, ak.num(A[event_idx]))
        all_masks.append(unflattened_mask)
        
    return ak.Array(all_masks)

class CCV1(Dataset):
    r'''
        input: layer clusters

    '''

    url = '/dummy/'

    def __init__(self, root, transform=None, max_events=1e8, inp = 'train'):
        super(CCV1, self).__init__(root, transform)
        self.step_size = 500
        self.inp = inp
        self.max_events = max_events
        self.fill_data(max_events)

    def fill_data(self,max_events):
        counter = 0
        arrLens0 = []
        arrLens1 = []

        print("### Loading data")
        for fi,path in enumerate(tqdm(self.raw_paths)):


            if self.inp == 'train':
                cluster_path = find_highest_branch(path, 'clusters')
                sim_path = find_highest_branch(path, 'simtrackstersCP')
            elif self.inp == 'val':
                cluster_path = find_highest_branch(path, 'clusters')
                sim_path = find_highest_branch(path, 'simtrackstersCP')
            else:
                cluster_path = find_highest_branch(path, 'clusters')
                sim_path = find_highest_branch(path, 'simtrackstersCP')
            
            crosstree =  uproot.open(path)[cluster_path]
            crosscounter = 0
            for array in uproot.iterate(f"{path}:{sim_path}", ["vertices_x", "vertices_y", "vertices_z", 
            "vertices_energy", "vertices_multiplicity", "vertices_time", "vertices_indexes", "barycenter_x", "barycenter_y", "barycenter_z"], step_size=self.step_size):
            
                tmp_stsCP_vertices_x = array['vertices_x']
                tmp_stsCP_vertices_y = array['vertices_y']
                tmp_stsCP_vertices_z = array['vertices_z']
                tmp_stsCP_vertices_energy = array['vertices_energy']
                tmp_stsCP_vertices_time = array['vertices_time']
                tmp_stsCP_vertices_indexes = array['vertices_indexes']
                tmp_stsCP_barycenter_x = array['barycenter_x']
                tmp_stsCP_barycenter_y = array['barycenter_y']
                tmp_stsCP_barycenter_z = array['barycenter_z']


                tmp_stsCP_vertices_multiplicity = array['vertices_multiplicity']
                
                # weighted energies (A LC appears in its caloparticle assignment array as the energy it contributes not full energy)
                #tmp_stsCP_vertices_energy = tmp_stsCP_vertices_energy * tmp_stsCP_vertices_multiplicity
                
                self.step_size = min(self.step_size,len(tmp_stsCP_vertices_x))


                # Code block for reading from other tree
                tmp_all_vertices_layer_id = crosstree['cluster_layer_id'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                #tmp_all_vertices_radius = crosstree['cluster_radius'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                tmp_all_vertices_noh = crosstree['cluster_number_of_hits'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                tmp_all_vertices_eta = crosstree['position_eta'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                tmp_all_vertices_phi = crosstree['position_phi'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                crosscounter += 1

                layer_id_list = []
                radius_list = []
                noh_list = []
                eta_list = []
                phi_list = []
                for evt_row in range(len(tmp_all_vertices_noh)):
                    #print("Event no: %i"%evt_row)
                    #print("There are %i particles in this event"%len(tmp_stsCP_vertices_indexes[evt_row]))
                    layer_id_list_one_event = []
                    #radius_list_one_event = []
                    noh_list_one_event = []
                    eta_list_one_event = []
                    phi_list_one_event = []
                    for particle in range(len(tmp_stsCP_vertices_indexes[evt_row])):
                        #print("Particle no: %i"%particle)
                        #print("A")
                        #print(np.array(tmp_all_vertices_radius[evt_row]).shape)
                        #print("B")
                        #print(np.array(tmp_stsCP_vertices_indexes[evt_row][particle]).shape)
                        #print("C")
                        tmp_stsCP_vertices_layer_id_one_particle = tmp_all_vertices_layer_id[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        #tmp_stsCP_vertices_radius_one_particle = tmp_all_vertices_radius[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        tmp_stsCP_vertices_noh_one_particle = tmp_all_vertices_noh[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        tmp_stsCP_vertices_eta_one_particle = tmp_all_vertices_eta[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        tmp_stsCP_vertices_phi_one_particle = tmp_all_vertices_phi[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        #print(tmp_stsCP_vertices_radius_one_particle)
                        layer_id_list_one_event.append(tmp_stsCP_vertices_layer_id_one_particle)
                        #radius_list_one_event.append(tmp_stsCP_vertices_radius_one_particle)
                        noh_list_one_event.append(tmp_stsCP_vertices_noh_one_particle)
                        eta_list_one_event.append(tmp_stsCP_vertices_eta_one_particle)
                        phi_list_one_event.append(tmp_stsCP_vertices_phi_one_particle)
                    layer_id_list.append(layer_id_list_one_event)
                    #radius_list.append(radius_list_one_event)
                    noh_list.append(noh_list_one_event)
                    eta_list.append(eta_list_one_event)
                    phi_list.append(phi_list_one_event)
                tmp_stsCP_vertices_layer_id = ak.Array(layer_id_list)                
                #tmp_stsCP_vertices_radius = ak.Array(radius_list)                
                tmp_stsCP_vertices_noh = ak.Array(noh_list)                
                tmp_stsCP_vertices_eta = ak.Array(eta_list)                
                tmp_stsCP_vertices_phi = ak.Array(phi_list)                
                
                # Apply filter noh > 1 for the LCs
                skim_mask_noh = tmp_stsCP_vertices_noh > 1.0
                tmp_stsCP_vertices_x = tmp_stsCP_vertices_x[skim_mask_noh]
                tmp_stsCP_vertices_y = tmp_stsCP_vertices_y[skim_mask_noh]
                tmp_stsCP_vertices_z = tmp_stsCP_vertices_z[skim_mask_noh]
                tmp_stsCP_vertices_energy = tmp_stsCP_vertices_energy[skim_mask_noh]
                tmp_stsCP_vertices_time = tmp_stsCP_vertices_time[skim_mask_noh]
                tmp_stsCP_vertices_layer_id = tmp_stsCP_vertices_layer_id[skim_mask_noh]
                #tmp_stsCP_vertices_radius = tmp_stsCP_vertices_radius[skim_mask_energyPercent]
                tmp_stsCP_vertices_noh = tmp_stsCP_vertices_noh[skim_mask_noh]
                tmp_stsCP_vertices_eta = tmp_stsCP_vertices_eta[skim_mask_noh]
                tmp_stsCP_vertices_phi = tmp_stsCP_vertices_phi[skim_mask_noh]
                #tmp_stsCP_vertices_indexes_unmasked = tmp_stsCP_vertices_indexes
                tmp_stsCP_vertices_indexes = tmp_stsCP_vertices_indexes[skim_mask_noh]
                tmp_stsCP_vertices_multiplicity = tmp_stsCP_vertices_multiplicity[skim_mask_noh]
                
                # Remove duplicates by only allowing the caloparticle that contributed the most energy to a LC to actually contribute.
                energyPercent = 1/tmp_stsCP_vertices_multiplicity
                skim_mask_energyPercent = remove_duplicates(tmp_stsCP_vertices_indexes,energyPercent)
                tmp_stsCP_vertices_x = tmp_stsCP_vertices_x[skim_mask_energyPercent]
                tmp_stsCP_vertices_y = tmp_stsCP_vertices_y[skim_mask_energyPercent]
                tmp_stsCP_vertices_z = tmp_stsCP_vertices_z[skim_mask_energyPercent]
                tmp_stsCP_vertices_energy = tmp_stsCP_vertices_energy[skim_mask_energyPercent]
                tmp_stsCP_vertices_time = tmp_stsCP_vertices_time[skim_mask_energyPercent]
                tmp_stsCP_vertices_layer_id = tmp_stsCP_vertices_layer_id[skim_mask_energyPercent]
                #tmp_stsCP_vertices_radius = tmp_stsCP_vertices_radius[skim_mask_energyPercent]
                tmp_stsCP_vertices_noh = tmp_stsCP_vertices_noh[skim_mask_energyPercent]
                tmp_stsCP_vertices_eta = tmp_stsCP_vertices_eta[skim_mask_energyPercent]
                tmp_stsCP_vertices_phi = tmp_stsCP_vertices_phi[skim_mask_energyPercent]
                #tmp_stsCP_vertices_indexes_unmasked = tmp_stsCP_vertices_indexes
                tmp_stsCP_vertices_indexes = tmp_stsCP_vertices_indexes[skim_mask_energyPercent]
                tmp_stsCP_vertices_multiplicity = tmp_stsCP_vertices_multiplicity[skim_mask_energyPercent]
               
                
                
                #SHOULD BE LEN(E) >= 2 for MULTI particles
                skim_mask = []
                for e in tmp_stsCP_vertices_x:
                    if 2 <= len(e) <= 5: #<------ only train on samples with > 1 particle
                        skim_mask.append(True)
                    else:
                        skim_mask.append(False)
                tmp_stsCP_vertices_x = tmp_stsCP_vertices_x[skim_mask]
                tmp_stsCP_vertices_y = tmp_stsCP_vertices_y[skim_mask]
                tmp_stsCP_vertices_z = tmp_stsCP_vertices_z[skim_mask]
                tmp_stsCP_vertices_energy = tmp_stsCP_vertices_energy[skim_mask]
                tmp_stsCP_vertices_time = tmp_stsCP_vertices_time[skim_mask]
                tmp_stsCP_vertices_layer_id = tmp_stsCP_vertices_layer_id[skim_mask]
                #tmp_stsCP_vertices_radius = tmp_stsCP_vertices_radius[skim_mask]
                tmp_stsCP_vertices_noh = tmp_stsCP_vertices_noh[skim_mask]
                tmp_stsCP_vertices_eta = tmp_stsCP_vertices_eta[skim_mask]
                tmp_stsCP_vertices_phi = tmp_stsCP_vertices_phi[skim_mask]
                #tmp_stsCP_vertices_indexes_unmasked = tmp_stsCP_vertices_indexes_unmasked[skim_mask]
                tmp_stsCP_vertices_indexes = tmp_stsCP_vertices_indexes[skim_mask]
                tmp_stsCP_vertices_multiplicity = tmp_stsCP_vertices_multiplicity[skim_mask]

                if counter == 0:
                    self.stsCP_vertices_x = tmp_stsCP_vertices_x
                    self.stsCP_vertices_y = tmp_stsCP_vertices_y
                    self.stsCP_vertices_z = tmp_stsCP_vertices_z
                    self.stsCP_vertices_energy = tmp_stsCP_vertices_energy
                    self.stsCP_vertices_time = tmp_stsCP_vertices_time
                    self.stsCP_vertices_layer_id = tmp_stsCP_vertices_layer_id
                    #self.stsCP_vertices_radius = tmp_stsCP_vertices_radius
                    self.stsCP_vertices_noh = tmp_stsCP_vertices_noh
                    self.stsCP_vertices_eta = tmp_stsCP_vertices_eta
                    self.stsCP_vertices_phi = tmp_stsCP_vertices_phi
                    self.stsCP_vertices_indexes = tmp_stsCP_vertices_indexes
                    #self.stsCP_vertices_indexes_unmasked = tmp_stsCP_vertices_indexes_unmasked
                    self.stsCP_barycenter_x = tmp_stsCP_barycenter_x
                    self.stsCP_barycenter_y = tmp_stsCP_barycenter_y
                    self.stsCP_barycenter_z = tmp_stsCP_barycenter_z
                    self.stsCP_vertices_multiplicity = tmp_stsCP_vertices_multiplicity
                else:
                    self.stsCP_vertices_x = ak.concatenate((self.stsCP_vertices_x,tmp_stsCP_vertices_x))
                    self.stsCP_vertices_y = ak.concatenate((self.stsCP_vertices_y,tmp_stsCP_vertices_y))
                    self.stsCP_vertices_z = ak.concatenate((self.stsCP_vertices_z,tmp_stsCP_vertices_z))
                    self.stsCP_vertices_energy = ak.concatenate((self.stsCP_vertices_energy,tmp_stsCP_vertices_energy))
                    self.stsCP_vertices_time = ak.concatenate((self.stsCP_vertices_time,tmp_stsCP_vertices_time))
                    self.stsCP_vertices_layer_id = ak.concatenate((self.stsCP_vertices_layer_id,tmp_stsCP_vertices_layer_id))
                    #self.stsCP_vertices_radius = ak.concatenate((self.stsCP_vertices_radius,tmp_stsCP_vertices_radius))
                    self.stsCP_vertices_noh = ak.concatenate((self.stsCP_vertices_noh,tmp_stsCP_vertices_noh))
                    self.stsCP_vertices_eta = ak.concatenate((self.stsCP_vertices_eta,tmp_stsCP_vertices_eta))
                    self.stsCP_vertices_phi = ak.concatenate((self.stsCP_vertices_phi,tmp_stsCP_vertices_phi))
                    self.stsCP_vertices_indexes = ak.concatenate((self.stsCP_vertices_indexes,tmp_stsCP_vertices_indexes))
                    #self.stsCP_vertices_indexes_unmasked =  ak.concatenate((self.stsCP_vertices_indexes_unmasked,tmp_stsCP_vertices_indexes_unmasked))
                    self.stsCP_barycenter_x = ak.concatenate((self.stsCP_barycenter_x,tmp_stsCP_barycenter_x))
                    self.stsCP_barycenter_y = ak.concatenate((self.stsCP_barycenter_y,tmp_stsCP_barycenter_y))
                    self.stsCP_barycenter_z = ak.concatenate((self.stsCP_barycenter_z,tmp_stsCP_barycenter_z))
                    self.stsCP_vertices_multiplicity = ak.concatenate((self.stsCP_vertices_multiplicity, tmp_stsCP_vertices_multiplicity))

                #print(len(self.stsCP_vertices_x))
                counter += 1
                if len(self.stsCP_vertices_x) > max_events:
                    print(f"Reached {max_events}!")
                    break
            if len(self.stsCP_vertices_x) > max_events:
                break
     
            
            
    def download(self):
        raise RuntimeError(
            'Dataset not found. Please download it from {} and move all '
            '*.z files to {}'.format(self.url, self.raw_dir))

    def len(self):
        return len(self.stsCP_vertices_x)

    @property
    def raw_file_names(self):
        raw_files = sorted(glob.glob(osp.join(self.raw_dir, '*.root')))
        
        #raw_files = [osp.join(self.raw_dir, 'step3_NTUPLE.root')]

        return raw_files

    @property
    def processed_file_names(self):
        return []


    def get(self, idx):
        def reconstruct_array(grouped_indices):
            # Find the maximum index to determine the array length
            max_index = max(max(indices) for indices in grouped_indices.values())

            # Initialize an array with the correct size, filled with a placeholder (e.g., -1)
            reconstructed = [-1] * (max_index + 1)

            # Populate the array based on the dictionary
            for value, indices in grouped_indices.items():
                for idx in indices:
                    reconstructed[idx] = value

            return reconstructed
        edge_index = torch.empty((2,0), dtype=torch.long)
 
        lc_x = self.stsCP_vertices_x[idx]
        #print(ak.to_numpy(lc_x[0]).shape)
        #print(ak.to_numpy(lc_x[1]).shape)
        flat_lc_x = np.expand_dims(np.array(ak.flatten(lc_x)),axis=1)
        lc_y = self.stsCP_vertices_y[idx]
        flat_lc_y = np.expand_dims(np.array(ak.flatten(lc_y)),axis=1)
        lc_z = self.stsCP_vertices_z[idx]
        flat_lc_z = np.expand_dims(np.array(ak.flatten(lc_z)),axis=1)
        lc_e = self.stsCP_vertices_energy[idx]
        flat_lc_e = np.expand_dims(np.array(ak.flatten(lc_e)),axis=1)     
        lc_t = self.stsCP_vertices_time[idx]
        flat_lc_t = np.expand_dims(np.array(ak.flatten(lc_t)),axis=1)  
        lc_layer_id = self.stsCP_vertices_layer_id[idx]
        flat_lc_layer_id = np.expand_dims(np.array(ak.flatten(lc_layer_id)),axis=1)  
        #lc_radius = self.stsCP_vertices_radius[idx]
        #flat_lc_radius = np.expand_dims(np.array(ak.flatten(lc_radius)),axis=1)  
        lc_noh = self.stsCP_vertices_noh[idx]
        flat_lc_noh = np.expand_dims(np.array(ak.flatten(lc_noh)),axis=1)  
        lc_eta = self.stsCP_vertices_eta[idx]
        flat_lc_eta = np.expand_dims(np.array(ak.flatten(lc_eta)),axis=1)  
        lc_phi = self.stsCP_vertices_phi[idx]
        flat_lc_phi = np.expand_dims(np.array(ak.flatten(lc_phi)),axis=1)  

        #lc_rad = self.stsCP_vertices_radius[idx]
        #flat_lc_rad = np.expand_dims(np.array(ak.flatten(lc_rad)),axis=1)  

        lc_indexes = self.stsCP_vertices_indexes[idx]
        #flat_lc_id = np.expand_dims(np.array(ak.flatten(lc_id)),axis=1)  
        lc_multiplicity = self.stsCP_vertices_multiplicity[idx]


        #flat_lc_feats = np.concatenate((flat_lc_x,flat_lc_y,flat_lc_z,flat_lc_e),axis=-1)
        flat_lc_feats = np.concatenate((flat_lc_x,flat_lc_y,flat_lc_z,flat_lc_e,\
                                        flat_lc_layer_id,flat_lc_noh,flat_lc_eta,flat_lc_phi),axis=-1)        

        # Loop over particles to build positive edges without self-pairs
        pos_edges = []
        idlc = 0
        offset = 0

        # Loop over particles (2-5 particles per event)
        for cp in range(len(lc_x)):
            n_lc_cp = len(lc_x[cp])

            for _ in range(n_lc_cp):
                # Randomly select an LC from the same particle, but not itself
                while True:
                    candidate = offset + int(random.uniform(0, n_lc_cp))
                    if candidate != idlc:
                        pos_edges.append([idlc, candidate])
                        break
                idlc += 1

            offset += n_lc_cp


        # Create the Data object
        x = torch.from_numpy(flat_lc_feats).float()
        x_lc = x
        x_pos_edge = torch.from_numpy(np.array(pos_edges).astype(np.int64))
        
        result = np.concatenate([np.full(len(subarr), i) for i, subarr in enumerate(lc_indexes)])
        result_list = result.tolist()                
        x_counts = lc_x
        y = torch.from_numpy(np.array([0 for u in range(len(flat_lc_feats))])).float()
        return Data(
            x=x_lc, edge_index=edge_index, x_pe = x_pos_edge, y=y,assoc = result_list )

In [3]:
ipath = "/vols/cms/mm1221/Data/100k/5e/train/"
vpath = "/vols/cms/mm1221/Data/100k/5e/val/"
data_train = CCV1(ipath, max_events=100, inp = 'train')
data_val = CCV1(vpath, max_events=100, inp='val')

### Loading data


  0%|                                                                                                                                                                                 | 0/3 [00:17<?, ?it/s]


Reached 100!
### Loading data


  0%|                                                                                                                                                                                 | 0/1 [00:17<?, ?it/s]

Reached 100!


In [40]:
train_sample = data_train[5].x  # shape: (num_nodes, 8)

# Compute per-feature statistics
feature_means = train_sample.mean(dim=0)
feature_stds = train_sample.std(dim=0)
mins, _ = train_sample.min(dim=0)
maxs, _ = train_sample.max(dim=0)

print("Feature shape:", train_sample.shape)
for i in range(train_sample.size(1)):
    print(f"Feature {i}: "
          f"mean={feature_means[i].item():.4f}, "
          f"std={feature_stds[i].item():.4f}, "
          f"min={mins[i].item():.4f}, "
          f"max={maxs[i].item():.4f}")


Feature shape: torch.Size([223, 8])
Feature 0: mean=-16.2156, std=5.2154, min=-27.4581, max=-0.7037
Feature 1: mean=47.8458, std=6.4625, min=32.2432, max=67.9368
Feature 2: mean=341.3482, std=8.7109, min=322.1550, max=374.2810
Feature 3: mean=1.1087, std=2.4231, min=0.0317, max=12.1245
Feature 4: mean=13.6323, std=5.4846, min=1.0000, max=28.0000
Feature 5: mean=7.6502, std=10.4443, min=2.0000, max=48.0000
Feature 6: mean=2.6164, std=0.1490, min=2.3293, max=3.0121
Feature 7: mean=1.8929, std=0.0806, min=1.5876, max=2.0832


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CustomStaticEdgeConv(nn.Module):
    def __init__(self, nn_module):
        super(CustomStaticEdgeConv, self).__init__()
        self.nn_module = nn_module

    def forward(self, x, edge_index):
        """
        Args:
            x (torch.Tensor): Node features of shape (N, F).
            edge_index (torch.Tensor): Predefined edges [2, E], where E is the number of edges.

        Returns:
            torch.Tensor: Node features after static edge aggregation.
        """
        row, col = edge_index  # Extract row (source) and col (target) nodes
        x_center = x[row]
        x_neighbor = x[col]

        # Compute edge features (relative)
        edge_features = torch.cat([x_center, x_neighbor - x_center], dim=-1)
        edge_features = self.nn_module(edge_features)

        # Aggregate features back to nodes
        num_nodes = x.size(0)
        node_features = torch.zeros(num_nodes, edge_features.size(-1), device=x.device)
        node_features.index_add_(0, row, edge_features)

        # Normalization (Divide by node degrees)
        counts = torch.bincount(row, minlength=num_nodes).clamp(min=1).view(-1, 1)
        node_features = node_features / counts

        return node_features


class CustomGATLayer(nn.Module):
    def __init__(self, in_dim, out_dim, heads=1, concat=True, dropout=0.6, alpha=0.4):
        """
        Initializes the Custom GAT Layer.

        Args:
            in_dim (int): Input feature dimension.
            out_dim (int): Output feature dimension per head.
            heads (int): Number of attention heads.
            concat (bool): Whether to concatenate the heads' output or average them.
            dropout (float): Dropout rate on attention coefficients.
            alpha (float): Negative slope for LeakyReLU.
        """
        super(CustomGATLayer, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.heads = heads
        self.concat = concat

        # Linear transformation for node features
        self.W = nn.Linear(in_dim, heads * out_dim, bias=False)

        # Attention mechanism: a vector for each head
        self.a_src = nn.Parameter(torch.zeros(heads, out_dim))
        self.a_tgt = nn.Parameter(torch.zeros(heads, out_dim))
        nn.init.xavier_uniform_(self.a_src.data, gain=1.414)
        nn.init.xavier_uniform_(self.a_tgt.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(alpha)
        self.dropout = nn.Dropout(dropout)

        # Optional batch normalization
        self.batch_norm = nn.BatchNorm1d(heads * out_dim) if concat else nn.BatchNorm1d(out_dim)

    def forward(self, x, edge_index):
        """
        Forward pass of the GAT layer.

        Args:
            x (torch.Tensor): Node features of shape (N, in_dim).
            edge_index (torch.Tensor): Edge indices of shape (2, E).

        Returns:
            torch.Tensor: Updated node features after attention-based aggregation.
        """
        src, tgt = edge_index  # Source and target node indices
        N = x.size(0)

        # Apply linear transformation and reshape for multi-head attention
        h = self.W(x)  # Shape: (N, heads * out_dim)
        h = h.view(N, self.heads, self.out_dim)  # Shape: (N, heads, out_dim)

        # Gather node features for each edge
        h_src = h[src]  # Shape: (E, heads, out_dim)
        h_tgt = h[tgt]  # Shape: (E, heads, out_dim)

        # Compute attention coefficients using separate vectors for source and target
        e_src = (h_src * self.a_src).sum(dim=-1)  # Shape: (E, heads)
        e_tgt = (h_tgt * self.a_tgt).sum(dim=-1)  # Shape: (E, heads)
        e = self.leakyrelu(e_src + e_tgt)  # Shape: (E, heads)

        # Compute softmax normalization for attention coefficients
        # To ensure numerical stability
        e = e - e.max(dim=0, keepdim=True)[0]
        alpha = torch.exp(e)  # Shape: (E, heads)

        # Sum of attention coefficients for each target node and head
        alpha_sum = torch.zeros(N, self.heads, device=x.device).scatter_add_(0, tgt.unsqueeze(-1).expand(-1, self.heads), alpha)

        # Avoid division by zero
        alpha_sum = alpha_sum + 1e-16

        # Normalize attention coefficients
        alpha = alpha / alpha_sum[tgt]  # Shape: (E, heads)
        alpha = self.dropout(alpha)

        # Weighted aggregation of source node features
        h_prime = h_src * alpha.unsqueeze(-1)  # Shape: (E, heads, out_dim)

        # Initialize output tensor and aggregate
        out = torch.zeros(N, self.heads, self.out_dim, device=x.device)
        out.scatter_add_(0, tgt.unsqueeze(-1).unsqueeze(-1).expand(-1, self.heads, self.out_dim), h_prime)  # Shape: (N, heads, out_dim)

        # Concatenate or average the heads
        if self.concat:
            out = out.view(N, self.heads * self.out_dim)  # Shape: (N, heads*out_dim)
        else:
            out = out.mean(dim=1)  # Shape: (N, out_dim)

        # Apply batch normalization
        out = self.batch_norm(out)

        return out

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import DynamicEdgeConv

class Net(nn.Module):
    def __init__(self, hidden_dim=64, num_layers=4, dropout=0.3, contrastive_dim=8, k=20):
        """
        Initializes the neural network with DynamicEdgeConv layers.

        Args:
            hidden_dim (int): Dimension of hidden layers.
            num_layers (int): Total number of DynamicEdgeConv layers.
            dropout (float): Dropout rate.
            contrastive_dim (int): Dimension of the contrastive output.
            k (int): Number of nearest neighbors to use in DynamicEdgeConv.
        """
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.contrastive_dim = contrastive_dim
        self.k = k

        # Input feature normalization (BatchNorm1d normalizes feature-wise across samples)
        self.input_norm = nn.BatchNorm1d(8)  # Normalizes across batch dimension

        # Input encoder
        self.lc_encode = nn.Sequential(
            nn.Linear(8, 32),
            nn.ELU(),
            nn.Linear(32, hidden_dim),
            nn.ELU()
        )

        # Define the network's convolutional layers using DynamicEdgeConv layers
        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            mlp = nn.Sequential(
                nn.Linear(2 * hidden_dim, hidden_dim),
                nn.ELU(),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(p=dropout)
            )
            conv = DynamicEdgeConv(mlp, k=k, aggr="max")
            self.convs.append(conv)

        # Output layer
        self.output = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ELU(),
            nn.Dropout(p=dropout),
            nn.Linear(64, 32),
            nn.ELU(),
            nn.Dropout(p=dropout),
            nn.Linear(32, contrastive_dim)
        )

    def forward(self, x, batch):
        """
        Forward pass of the network.

        Args:
            x (torch.Tensor): Input node features of shape (N, 8).
            batch (torch.Tensor): Batch vector that assigns each node to an example in the batch.

        Returns:
            torch.Tensor: Output features after processing.
            torch.Tensor: Batch vector.
        """
        # Normalize input features
        x = self.input_norm(x)

        # Input encoding
        x_lc_enc = self.lc_encode(x)  # Shape: (N, hidden_dim)

        # Apply DynamicEdgeConv layers with residual connections
        feats = x_lc_enc
        for conv in self.convs:
            feats = conv(feats, batch) 

        # Final output
        out = self.output(feats)
        return out, batch





In [5]:
# Initialize model with passed hyperparameters
model = Net(
    hidden_dim=64,
    num_layers=3,
    dropout=0.3,
    contrastive_dim=128,
    k=16
)

k_value = 4
BS = 1
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)


# Define Loss

In [24]:
def contrastive_loss_random_pos_and_per_group_neg(
    embeddings,
    group_ids,
    temperature=0.1
):
    """
    Computes an InfoNCE-style loss where for each anchor:
      - We randomly select ONE positive from the same group (excluding itself).
      - We randomly select ONE negative from each other group.

    Args:
        embeddings: (N, D) tensor of raw embedding outputs (will be normalized internally).
        group_ids: (N,) 1D tensor of group identifiers (e.g. [0, 0, 1, 1, 2, 2, ...]).
        temperature: Scalar temperature for the softmax.

    Returns:
        Scalar loss (mean over anchors).
    """
    # ---------------------------
    # Setup and precompute
    # ---------------------------
    N = embeddings.size(0)
    device = embeddings.device

    # Normalize embeddings so that cosine similarity is simply the dot product
    norm_emb = F.normalize(embeddings, p=2, dim=1)  # shape (N, D)
    # Precompute similarity matrix
    sim_matrix = norm_emb @ norm_emb.t()  # (N, N)

    # Group -> list of indices in that group
    group2indices = {}
    for i, g in enumerate(group_ids.tolist()):
        group2indices.setdefault(g, []).append(i)


    unique_groups = sorted(group2indices.keys())


    losses = []

    # ---------------------------
    # For each anchor, sample pos + (one) neg from each other group
    # ---------------------------
    for i in range(N):
        anchor_group = group_ids[i].item()
        anchor_sim = sim_matrix[i]  # similarities to all others
        print(anchor_group)
        print(anchor_sim)
        # Gather candidate positives in same group (excluding itself)
        same_group_indices = group2indices[anchor_group]
        if len(same_group_indices) < 2:
            # If there's no valid alternative positive in the same group, skip or count zero
            losses.append(torch.tensor(0.0, device=device))
            continue

        # Randomly choose one positive different from i
        valid_pos_indices = [idx for idx in same_group_indices if idx != i]
        pos_index = int(torch.randint(low=0, high=len(valid_pos_indices), size=(1,)))
        pos_index = valid_pos_indices[pos_index]  # actual index in [0..N-1]

        sim_pos = anchor_sim[pos_index]  # anchor->positive similarity
        print('pos',pos_index)

        # Now get one random negative from each other group
        neg_sims = []
        for g in unique_groups:
            if g == anchor_group:
                continue  # skip own group
            # random negative from group g
            neg_candidates = group2indices[g]
            neg_idx = int(torch.randint(low=0, high=len(neg_candidates), size=(1,)))
            neg_idx = neg_candidates[neg_idx]
            neg_sims.append(anchor_sim[neg_idx])
            print('ni',neg_idx)
            print('nsims',neg_sims)

        # If for some reason no negatives, skip or zero
        if len(neg_sims) == 0:
            losses.append(torch.tensor(0.0, device=device))
            continue

        # Compute denominator: exp(pos / T) + sum of exp(neg / T) for each negative
        numerator = torch.exp(sim_pos / temperature)
        denominator = numerator + torch.sum(torch.exp(torch.stack(neg_sims) / temperature))

        # InfoNCE
        loss_i = -torch.log(numerator / denominator)
        print(loss_i)
        losses.append(loss_i)

    # Mean over anchors
    return torch.stack(losses).mean()

def contrastive_loss_curriculum(embeddings, pos_indices, group_ids, temperature=0.1, alpha=1.0):
    """
    Curriculum loss that uses both positive and negative blending.
    
    Delegates to contrastive_loss_curriculum_both.
    
    Args:
        embeddings: Tensor of shape (N, D).
        pos_indices: 1D Tensor (length N).
        group_ids: 1D Tensor (length N).
        temperature: Temperature scaling factor.
        alpha: Blending parameter.
        
    Returns:
        Scalar loss.
    """
    return contrastive_loss_random_pos_and_per_group_neg(embeddings, group_ids, temperature)

In [25]:
def train_new(train_loader, model, optimizer, device, k_value, alpha):
    model.train()
    total_loss = torch.zeros(1, device=device)
    for data in tqdm(train_loader, desc="Training"):
        data = data.to(device)
        optimizer.zero_grad()
        
        # Convert data.assoc to tensor if needed.
        if isinstance(data.assoc, list):
            if isinstance(data.assoc[0], list):
                assoc_tensor = torch.cat([torch.tensor(a, dtype=torch.int64, device=data.x.device)
                                          for a in data.assoc])
            else:
                assoc_tensor = torch.tensor(data.assoc, device=data.x.device)
        else:
            assoc_tensor = data.assoc

        #edge_index = knn_graph(data.x[:, :3], k=k_value, batch=data.x_batch)
        embeddings, _ = model(data.x, data.x_batch)
        
        # Partition batch by event.
        batch_np = data.x_batch.detach().cpu().numpy()
        _, counts = np.unique(batch_np, return_counts=True)
        
        loss_event_total = torch.zeros(1, device=device)
        start_idx = 0
        for count in counts:
            end_idx = start_idx + count
            event_embeddings = embeddings[start_idx:end_idx]
            event_group_ids = assoc_tensor[start_idx:end_idx]

            event_pos_indices = data.x_pe[start_idx:end_idx, 1].view(-1)

            loss_event = contrastive_loss_curriculum(event_embeddings, event_pos_indices,
                                                     event_group_ids, temperature=0.1, alpha=alpha)
            loss_event_total += loss_event
            start_idx = end_idx
        
        loss = loss_event_total / len(counts)
        loss.backward()
        total_loss += loss
        optimizer.step()
    return total_loss / len(train_loader.dataset)

@torch.no_grad()
def test_new(test_loader, model, device, k_value, alpha):
    model.eval()
    total_loss = torch.zeros(1, device=device)
    for data in tqdm(test_loader, desc="Validation"):
        data = data.to(device)
        
        if isinstance(data.assoc, list):
            if isinstance(data.assoc[0], list):
                assoc_tensor = torch.cat([torch.tensor(a, dtype=torch.int64, device=data.x.device)
                                          for a in data.assoc])
            else:
                assoc_tensor = torch.tensor(data.assoc, device=data.x.device)
        else:
            assoc_tensor = data.assoc
        
        #edge_index = knn_graph(data.x[:, :3], k=k_value, batch=data.x_batch)
        embeddings, _ = model(data.x, data.x_batch)
        
        batch_np = data.x_batch.detach().cpu().numpy()
        _, counts = np.unique(batch_np, return_counts=True)
        
        loss_event_total = torch.zeros(1, device=device)
        start_idx = 0
        for count in counts:
            end_idx = start_idx + count
            event_embeddings = embeddings[start_idx:end_idx]
            event_group_ids = assoc_tensor[start_idx:end_idx]

            event_pos_indices = data.x_pe[start_idx:end_idx, 1].view(-1)
            loss_event = contrastive_loss_curriculum(event_embeddings, event_pos_indices,
                                                     event_group_ids, temperature=0.1, alpha=alpha)
            loss_event_total += loss_event
            start_idx = end_idx
        total_loss += loss_event_total / len(counts)
    return total_loss / len(test_loader.dataset)


In [26]:

device = torch.device('cpu')
# Load DataLoader with current batch_size
train_loader = DataLoader(data_train, batch_size=BS, shuffle=True, follow_batch=['x'])
val_loader = DataLoader(data_val, batch_size=BS, shuffle=False, follow_batch=['x'])

# Train and evaluate the model for the specified number of epochs
best_val_loss = float('inf')

# Store train and validation losses for all epochs
train_losses = []
val_losses = []

output_dir = '/vols/cms/mm1221/hgcal/elec5New/LC/multi/resultstest/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
patience =30    
epochs = 5
for epoch in range(epochs):
    # For epochs 1 to 150, gradually increase alpha from 0 to 1.
    # From epoch 151 onward, set alpha = 1 (fully hard negatives).
    alpha = 0

    print(f"Epoch {epoch+1}/{epochs} | Alpha: {alpha:.2f}")
    train_loss = train_new(train_loader, model, optimizer, device, k_value, alpha)
    val_loss = test_new(val_loader, model, device, k_value, alpha)

    train_losses.append(train_loss.item())
    val_losses.append(val_loss.item())
    scheduler.step()

    # Save best model if validation loss improves.
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement_epochs = 0
        torch.save(model.state_dict(), os.path.join(output_dir, 'best_model.pt'))
    else:
        no_improvement_epochs += 1

    # Save intermediate checkpoint.
    state_dicts = {'model': model.state_dict(),
                   'opt': optimizer.state_dict(),
                   'lr': scheduler.state_dict()}
    torch.save(state_dicts, os.path.join(output_dir, f'epoch-{epoch+1}.pt'))

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss.item():.8f}, Validation Loss: {val_loss.item():.8f}")
    if no_improvement_epochs >= patience:
        print(f"Early stopping triggered. No improvement for {patience} epochs.")
        break

# Save training history.
import pandas as pd
results_df = pd.DataFrame({
    'epoch': list(range(1, len(train_losses) + 1)),
    'train_loss': train_losses,
    'val_loss': val_losses
})
results_df.to_csv(os.path.join(output_dir, 'continued_training_loss.csv'), index=False)
print(f"Saved loss curves to {os.path.join(output_dir, 'continued_training_loss.csv')}")

# Save final model.
torch.save(model.state_dict(), os.path.join(output_dir, 'final_model.pt'))
print("Training complete. Final model saved.")

Epoch 1/5 | Alpha: 0.00


Training:   0%|                                                                                                                                                                     | 0/418 [00:00<?, ?it/s]

0
tensor([ 1.0000e+00,  6.1751e-01,  5.2279e-01,  6.0217e-01,  3.2712e-01,
         5.0323e-01,  7.3585e-01,  6.3428e-01,  4.0560e-01,  6.4161e-01,
         4.3557e-01,  6.8904e-01,  3.5884e-01,  5.1395e-01,  6.2651e-01,
         5.5404e-01,  5.3178e-01,  5.3620e-01,  7.0919e-01,  5.3457e-01,
         4.8820e-01,  6.0108e-01,  2.6227e-01,  5.9190e-01,  4.9186e-01,
         4.5023e-01,  5.7845e-01,  5.7116e-01,  5.9189e-01,  4.3873e-02,
         5.6439e-01,  5.4966e-01,  5.4647e-01,  2.9395e-01,  7.4194e-01,
         5.0583e-01,  6.8742e-01,  4.1256e-01,  6.0420e-01,  4.0684e-01,
         5.4809e-01,  6.6810e-01,  3.6237e-01,  4.2387e-01,  5.7797e-01,
         5.4675e-01,  5.3754e-01,  5.8196e-01,  3.5858e-01,  4.1082e-01,
         2.8468e-01,  4.7824e-01,  5.3483e-01,  5.0787e-01,  5.7405e-01,
         1.8892e-02,  4.6138e-01,  4.0749e-01,  5.7792e-01,  4.2577e-01,
         5.2740e-01,  5.5731e-01,  5.9579e-01,  1.3639e-01,  4.1562e-01,
         5.5287e-01,  4.6751e-01,  6.0278e-01,  4

nsims [tensor(0.4374, grad_fn=<SelectBackward0>), tensor(0.4505, grad_fn=<SelectBackward0>)]
tensor(1.5279, grad_fn=<NegBackward0>)
0
tensor([ 0.5780,  0.6404,  0.5510,  0.6197,  0.2549,  0.6200,  0.5992,  0.3604,
         0.5843,  0.6758,  0.5708,  0.6609,  0.3586,  0.6195,  0.6446,  0.6926,
         0.6781,  0.6900,  0.7366,  0.5817,  0.5873,  0.6397,  0.3629,  0.7221,
         0.6133,  0.5686,  0.7010,  0.7411,  0.6768,  0.2138,  0.6896,  0.6330,
         0.6748,  0.4692,  0.6560,  0.5802,  0.5853,  0.4577,  0.5817,  0.5552,
         0.4965,  0.6279,  0.6466,  0.5212,  1.0000,  0.7125,  0.6901,  0.7382,
         0.7120,  0.5549,  0.4675,  0.7031,  0.7561,  0.6854,  0.8775,  0.2020,
         0.5410,  0.7082,  0.6753,  0.5415,  0.7181,  0.6450,  0.6229,  0.5104,
         0.7188,  0.7335,  0.6347,  0.8726,  0.6987,  0.3609,  0.7890,  0.3660,
         0.6609,  0.7550,  0.6367,  0.7843,  0.4498,  0.7952,  0.4191,  0.6407,
         0.6731,  0.7251,  0.6023,  0.5417,  0.4881,  0.6280,  0.4

nsims [tensor(0.2167, grad_fn=<SelectBackward0>), tensor(0.5147, grad_fn=<SelectBackward0>)]
tensor(0.3805, grad_fn=<NegBackward0>)
0
tensor([ 0.0189,  0.2554,  0.3154,  0.1184,  0.1072,  0.0944,  0.1763,  0.1689,
         0.3285,  0.3033,  0.2541,  0.2737,  0.4106,  0.1544,  0.3752,  0.1628,
         0.1776,  0.3359,  0.2940,  0.3322,  0.1688, -0.0438,  0.2846,  0.3217,
         0.4152,  0.2735,  0.1647,  0.3050,  0.2106, -0.0467,  0.0710,  0.2421,
         0.0852,  0.0703,  0.2012,  0.3934,  0.3162,  0.2438,  0.0759,  0.0667,
         0.1196,  0.3029,  0.2795,  0.2510,  0.2020,  0.2616,  0.3223, -0.1199,
         0.1308,  0.3539,  0.0681, -0.0417,  0.2699,  0.1235,  0.1201,  1.0000,
         0.3192, -0.1031,  0.2928,  0.3611,  0.1642,  0.2575,  0.1683,  0.0573,
         0.4067,  0.2678,  0.1388,  0.2300,  0.0531, -0.0795,  0.0684,  0.3863,
         0.1916,  0.0611,  0.3282,  0.0777, -0.1860,  0.3376,  0.3761,  0.3411,
         0.3495,  0.0906,  0.1814,  0.1632,  0.1006, -0.1350,  0.0

...

ni 133
nsims [tensor(0.5627, grad_fn=<SelectBackward0>), tensor(0.3171, grad_fn=<SelectBackward0>)]
ni 148
nsims [tensor(0.5627, grad_fn=<SelectBackward0>), tensor(0.3171, grad_fn=<SelectBackward0>), tensor(0.5521, grad_fn=<SelectBackward0>)]
tensor(1.0845, grad_fn=<NegBackward0>)
3
tensor([ 0.2576,  0.3153,  0.5233,  0.5835,  0.4368,  0.5845,  0.7074,  0.7136,
         0.5229,  0.6524,  0.4988,  0.4146,  0.5366,  0.5240,  0.5263,  0.5703,
         0.4913,  0.5191,  0.3570,  0.3789,  0.3234,  0.1917,  0.6077,  0.3900,
         0.4453,  0.6435,  0.7302,  0.5285,  0.4984,  0.6204,  0.6793,  0.6433,
         0.6456,  0.4755,  0.4609,  0.6197,  0.5453,  0.5546,  0.5239,  0.5927,
         0.7764,  0.5002,  0.4710,  0.7095,  0.6758,  0.5341,  0.4932,  0.6479,
         0.4035,  0.4628,  0.5728,  0.0818,  0.5235,  0.5677,  0.6853,  0.8332,
         0.5723,  0.7308,  0.5009,  0.5468,  0.5592,  0.5136,  0.4052,  0.4735,
         0.5089,  0.5468,  0.5902,  0.7667,  0.7142,  0.6625,  0.7126,  0.40

nsims [tensor(0.6147, grad_fn=<SelectBackward0>), tensor(0.4580, grad_fn=<SelectBackward0>)]
ni 204
nsims [tensor(0.6147, grad_fn=<SelectBackward0>), tensor(0.4580, grad_fn=<SelectBackward0>), tensor(0.5176, grad_fn=<SelectBackward0>)]
tensor(0.2607, grad_fn=<NegBackward0>)
3
tensor([ 0.2331,  0.4145,  0.4254,  0.3428,  0.3756,  0.2734,  0.6979,  0.6255,
         0.4131,  0.4839,  0.3750,  0.4680,  0.4823,  0.5175,  0.2700,  0.3230,
         0.3572,  0.4737,  0.3294,  0.4981,  0.4378,  0.4086,  0.5558,  0.3273,
         0.6636,  0.5911,  0.6109,  0.4702,  0.4880,  0.3995,  0.5877,  0.5973,
         0.6602,  0.3590,  0.5762,  0.4538,  0.5056,  0.5200,  0.4765,  0.5377,
         0.6386,  0.4547,  0.3915,  0.6661,  0.6299,  0.3962,  0.7897,  0.6264,
         0.5746,  0.4470,  0.5548,  0.3057,  0.3369,  0.4093,  0.5897,  0.7748,
         0.4637,  0.6656,  0.3795,  0.4293,  0.4879,  0.4865,  0.4314,  0.4730,
         0.5376,  0.3364,  0.5261,  0.4979,  0.4483,  0.5572,  0.4714,  0.4369,
   

Training:   0%|▍                                                                                                                                                            | 1/418 [00:02<16:28,  2.37s/it]

[tensor(0.5615, grad_fn=<SelectBackward0>), tensor(0.6784, grad_fn=<SelectBackward0>)]
ni 144
nsims [tensor(0.5615, grad_fn=<SelectBackward0>), tensor(0.6784, grad_fn=<SelectBackward0>), tensor(0.5170, grad_fn=<SelectBackward0>)]
tensor(1.1626, grad_fn=<NegBackward0>)
3
tensor([ 0.3922,  0.1674,  0.4142,  0.3103,  0.1793,  0.2678,  0.3379,  0.2629,
         0.4092,  0.2931,  0.0404,  0.1547,  0.2121,  0.3554,  0.2293,  0.0952,
         0.1175,  0.4960,  0.3717,  0.4423,  0.1048,  0.1986,  0.5261,  0.3881,
         0.5098,  0.3741,  0.4210,  0.3628,  0.4670,  0.3398,  0.5352,  0.4517,
         0.2529,  0.2555,  0.4921,  0.2817,  0.4713,  0.3870,  0.2231,  0.4431,
         0.3925,  0.3356,  0.4288,  0.4800,  0.5849,  0.1816,  0.4974,  0.4687,
         0.3800,  0.3499,  0.5262,  0.3552,  0.2484,  0.1802,  0.4455,  0.4516,
         0.4871,  0.5201,  0.4059,  0.3530,  0.3533,  0.3581,  0.4410,  0.2449,
         0.2093,  0.3708,  0.3153,  0.3091,  0.5198,  0.5460,  0.3369,  0.5814,
         

KeyboardInterrupt: 

In [ ]:
print(data_train[0].assoc)